In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

Scrape from this site the 100 best production companies since we did not find any dataset for this.

In [ ]:
url = "https://www.the-numbers.com/movies/production-companies"

# --- UPDATED: Define a more complete set of browser headers ---
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Referer': 'https://www.google.com/', # Pretend we came from Google
}

print("Sending request with advanced User-Agent and headers...")
try:
    # --- Good practice: Wait briefly before the request ---
    time.sleep(1) 
    
    response = requests.get(url, headers=headers)
    response.raise_for_status() 
    print("Request successful (Status 200). Parsing HTML...")

    # Parse the HTML code using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    companies = []
    budgets = []
    
    # Logic to find the table and companies (already proven correct)
    table = soup.find('table')

    if table:
        for row in table.find_all('tr')[1:]:
            cols = row.find_all('td')
            company_cell = cols[0]
            link = company_cell.find('a')
            title = link.get_text().strip()
            total_gross = cols[3].get_text().strip()
            companies.append(title)
            budgets.append(total_gross)
                
    
    print("\n--- Scraped Company Names (First 10) ---")
    if companies:
        for name in companies[:10]:
            print(name)
        print(f"\nTotal companies scraped: {len(companies)}")
    else:
        print("Scraping finished, but no company names were found. The HTML structure may have changed.")

except requests.exceptions.HTTPError as err:
    print(f"\nFailed with HTTP Error: {err}")
    print("This indicates the server continues to block automated requests despite advanced headers.")
except Exception as e:
    print(f"\nAn unexpected error occurred: {e}")

In [ ]:
df_productions = pd.DataFrame({
    'company': companies,
    'gross': budgets  # Use 'gross' initially, we will rename later if needed
})

df_productions['gross'] = df_productions['gross'].astype(str).str.replace(r'[$,]', '', regex=True)

# Convert the resulting clean string column to a numeric float
# 'errors="coerce"' will turn any non-convertible strings into NaN
df_productions['gross'] = pd.to_numeric(df_productions['gross'], errors='coerce')
df_productions = df_productions.sort_values(by='gross', ascending=False)

# 2. Save the sorted DataFrame to a CSV file
# index=False prevents pandas from writing the DataFrame's index as a column in the CSV
df_productions

In [ ]:
df_sorted.to_csv(r'C:\Users\elias\Downloads\Data Management\highest_to_lowest_gross.csv', index=False)